<a href="https://colab.research.google.com/github/SezerBugday/photogrammetry_meshroom_colab_flask/blob/main/Bitirme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 main.py

In [20]:
from pyngrok import ngrok

import os  
import shutil  


from flask import Flask, flash, request, redirect, url_for,render_template
from werkzeug.utils import secure_filename

port_no = 5000

import sqlite3 as sql


con = sql.connect("/content/photogrammetry_meshroom_colab_flask/sezerdb.db", check_same_thread=False)
cursor = con.cursor()

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import sys

import random


In [36]:
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

TEMPLATE = "/content/photogrammetry_meshroom_colab_flask/templates"
app = Flask(__name__, template_folder=TEMPLATE)
app._static_folder = '/content/static/'

#bootsrapt = Bootstrap(app)
islem_durumu=False;


app.config['UPLOAD_FOLDER'] = "/content/kirli"

ngrok.set_auth_token("2DJyoS7QMsne6ZsjPBWlik5uBBv_37hajTNHz6NFWXeczXf7A")
public_url = ngrok.connect(port_no).public_url

print(f"To acces the Gloable link please click {public_url}")


@app.route("/")
def main_page():

    print("Anasayfaya donuldu")
    return render_template("HomePage.html")

@app.route("/uploadimages", methods=["GET", "POST"])
def upload_file():
    print("resimler secildi")
    if request.method == 'POST':
        files = request.files.getlist("image")
        for file in files:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], file.filename))
            print("Görseller kaydedildi")
            return redirect(url_for('Show_Process'))
    return render_template("ShowSelectedImages.html")
  
@app.route("/process")
def Show_Process():
  if(islem_durumu==False):
    return redirect(url_for('Process'))
  return render_template("processing.html")
  print("isleniyor")

  
@app.route("/sezer")
def Process():
  return render_template("showModel.html")
  return "çalisti"
  os.system("rembg p /content/kirli   /content/temiz")
  os.system("./meshroom/Meshroom-2019.1.0/meshroom_photogrammetry --input /content/temiz --output /content/modelFolder")
  os.system("obj2gltf -i /content/modelFolder/model.obj -o /content/static/model.glb")
  
@app.route("/SingUp", methods=["GET", "POST"])
def SingUp():
   if request.method == 'POST':
        name = request.form.get("name")
        surname = request.form.get("surname")
        global email
      
        email = request.form.get("email")
        password = request.form.get("password")
        print("before db")
        print(name)
        #cursor.execute("INSERT INTO Users VALUES(name,surname,password,email)")
        cursor.execute("insert into Users values(?,?,?,?,?,?)",(name, surname, password,email,0,None))
        con.commit()
        global user_id
        cursor.execute("SELECT CustomerID  FROM  Users WHERE Email = ? and Password = ? ", (user_mail, user_password))
        veri = cursor.fetchall()
        user_id= veri[1][0]
        print(veri[1][0])
        print("after db")
        # Gmail email sunucusuna bağlanıyoruz
        try:
            global ver_code
            ver_code = random.randint(100000,999999)
            print("vercode", ver_code)
            mail = smtplib.SMTP("smtp.gmail.com",587)
            mail.ehlo()
            mail.starttls()
            mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")
            mesaj = MIMEMultipart()
            mesaj["From"] = "3D AR Object Platform "           # Gönderen
            mesaj["Subject"] = "Verification Code"   # Konusu
            mesaj["To"] = email
            body = "Your Verification Code :  "+ str(ver_code) 
            body_text = MIMEText(body, "plain")  #
            mesaj.attach(body_text)
            mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
            print("Mail başarılı bir şekilde gönderildi.")
            mail.close()
            return render_template("SentVerificationCode.html")
        # Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
        except:
            print("Hata:", sys.exc_info()[0])
            
   return render_template("SingUp.html")

@app.route("/SentCode", methods=["GET", "POST"])
def SentCode():
  if request.method == 'POST':
    code = request.form.get("sifre")
    print("gelen kod", ver_code)
    print("girilen kod", code)

    if(int(code) == ver_code):
      cursor.execute("UPDATE Users SET Is_Active = 1 WHERE Email = email" )
      con.commit()
      return render_template("Login.html",yazi="Tebrikler mailiniz onaylandı şimdi giriş yapabilirsiniz.")
    else:
      return render_template("SentVerificationCode.html",yazi="Mail onaylanmadı kodu tekrar giriniz.")
  return render_template("SentVerificationCode.html")

  
@app.route("/CheckLogin", methods=["GET", "POST"])
def CheckLogin():
  if request.method == 'POST':
    user_mail = request.form.get("email")
    user_password = request.form.get("password")
    print("Check",user_mail)
    print("Check2",user_password)
    cursor.execute("SELECT * FROM  Users WHERE Email = ? and Password = ? ", (user_mail, user_password))
    veri = cursor.fetchall() 
    if len(veri) != 0:
      return render_template("AddProduct.html")
    else:
      return render_template("Login.html",yazi="Kullanıcı maili veya şifre yanlış tekrar deneyiniz")
  
@app.route("/SaveProduct", methods=["GET", "POST"])
def SaveProduct():
  if request.method == 'POST':
    product_name = request.form.get("product_name")
    product_info = request.form.get("product_info")
    user_place1 = request.form.get("place1")
    user_place2= request.form.get("place2")
    user_place3 = request.form.get("place3")

    cursor.execute("insert into Products values(?,?,?,?,?,?,?,?)",(product_name,user_id,None, product_info, None,user_place1,user_place2,user_place3))
    con.commit()
    
    return kaydedildi
   
      

       
if __name__ == "__main__":
    app.run()

To acces the Gloable link please click http://d593-35-243-217-246.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 19:26:12] "GET / HTTP/1.1" 200 -


Anasayfaya donuldu


INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 19:26:15] "GET /uploadimages HTTP/1.1" 200 -


resimler secildi


...

INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 19:26:55] "POST /SentCode HTTP/1.1" 200 -


gelen kod 832732
girilen kod 832732


INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 19:27:05] "POST /CheckLogin HTTP/1.1" 200 -


Check sezerbugday2@gmail.com
Check2 123456


ERROR:__main__:Exception on /SaveProduct [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-36-c6660d6e5976>", line 141, in SaveProduct
    return kaydedildi
NameError: name 'kaydedildi' is not de

In [ ]:


# Gmail email sunucusuna bağlanıyoruz
try:
    mail = smtplib.SMTP("smtp.gmail.com",587)
    mail.ehlo()
    mail.starttls()
    mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")

    mesaj = MIMEMultipart()
    mesaj["From"] = "3D AR Object Platform "           # Gönderen
    mesaj["Subject"] = "Verification Code"    # Konusu
    mesaj["To"] = "sezerbugday2@gmail.com "
    body = " Bu mesaj sezer tarafından python ile gönderilmiştir "

    body_text = MIMEText(body, "plain")  #
    mesaj.attach(body_text)

    mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
    print("Mail başarılı bir şekilde gönderildi.")
    mail.close()

# Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
except:
    print("Hata:", sys.exc_info()[0])

Mail başarılı bir şekilde gönderildi.


In [37]:
cursor.execute("Select * From Products")
veri = cursor.fetchall() 
print(veri)

[('kup', 1, None, None, None, None, None, None), ('urun1', 45, None, 'urun2', None, 'n11', 'hepsiburada', 'trendyol')]


In [ ]:
cursor.execute("insert into Users values(?,?,?,?,?,?)",("name", "surname", "123456","sezerbugday2@gmail.com",0,None))
con.commit()

In [23]:
cursor.execute("SELECT * FROM  Users WHERE Email = 'sezerbugday2@gmail.com' and Password = 123456")
veri = cursor.fetchall() 
print(veri)
print(len(veri))

[('name', 'surname', '123456', 'sezerbugday2@gmail.com', 1, 35), ('dsfd', 'sdfdsf', '123456', 'sezerbugday2@gmail.com', 1, 40), ('fsddfs', 'sdfsdfds', '123456', 'sezerbugday2@gmail.com', 1, 43)]
3


In [31]:
def VeriSil():
    cursor.execute("DELETE FROM Users")
    con.commit()
VeriSil()

In [28]:
user_mail = 'sezerbugday2@gmail.com'
user_password= '123456'
cursor.execute("SELECT CustomerID  FROM  Users WHERE Email = ? and Password = ? ", (user_mail, user_password))
veri = cursor.fetchall() 
print(veri[1][0])

40


In [9]:
%cd /content/photogrammetry_meshroom_colab_flask

/content/photogrammetry_meshroom_colab_flask


In [11]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   sezerdb.db
	modified:   templates/HomePage.html

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	templates/Profil.html

no changes added to commit (use "git add" and/or "git commit -a")


In [12]:
!git add --all  

In [14]:
!git config --global user.email "sezerbugday2@gmail.com"

In [15]:
!git commit -a -m "urun tablosu eklendi"


[main 8120091] urun tablosu eklendi
 3 files changed, 371 insertions(+), 53 deletions(-)
 rewrite templates/HomePage.html (93%)
 create mode 100644 templates/Profil.html


In [16]:
!git branch

* main


In [17]:
!git push origin main

Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 3.85 KiB | 3.85 MiB/s, done.
Total 6 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/SezerBugday/photogrammetry_meshroom_colab_flask.git
   c1bc377..8120091  main -> main
